In [629]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']
def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def get_f1accs(df: Union[pd.DataFrame, List[pd.DataFrame]]):
    metrics = ['Vanilla test acc','Vanilla test f1','Shadow test acc','Shadow test f1','MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Vanilla runtime per']
    if isinstance(df,list):
        res = []
        for i in df:
            res.append(i[metrics].max())
        return res
    else:
        return df[metrics].max()
    

total_parms_new.columns

Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

In [630]:
def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==False)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==False)&(df['Rdp']==False)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

In [631]:
clean_adam = total_parms[(total_parms['Dp']==False)&(total_parms['Rdp']==False)&(total_parms['Ldp']==False)&(total_parms['Optim type']=='adam')]
clean_sgd = total_parms[(total_parms['Dp']==False)&(total_parms['Rdp']==False)&(total_parms['Ldp']==False)&(total_parms['Optim type']=='sgd')]


In [632]:
clean_sgd

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type


In [633]:
_,clean_sgd = baseline(total_parms_new)

In [634]:
clean_sgd

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type
510,cora,0.8933,0.5738,0.8933,0.7800,0.8644,0.7800,0.8916,0.6144,0.8916,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
511,cora,0.9031,0.6010,0.9031,0.8013,0.7987,0.8013,0.8916,0.5872,0.8916,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
512,cora,0.8785,0.6489,0.8785,0.7931,0.8245,0.7931,0.8736,0.6153,0.8736,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
513,cora,0.8818,0.6372,0.8818,0.7898,0.8272,0.7898,0.9015,0.5717,0.9015,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
514,cora,0.8851,0.6160,0.8851,0.8079,0.8025,0.8079,0.8785,0.6212,0.8785,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,flickr,0.5112,1.3820,0.5112,0.4486,1.5790,0.4486,0.5131,1.3889,0.5131,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
853,flickr,0.5144,1.3810,0.5144,0.4596,1.5552,0.4596,0.5182,1.3805,0.5182,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
854,flickr,0.5118,1.3960,0.5118,0.4561,1.5663,0.4561,0.5117,1.4054,0.5117,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd
855,flickr,0.5235,1.3671,0.5235,0.4555,1.5888,0.4555,0.5078,1.3890,0.5078,...,0.45,2,256,0.02,0.02,0.5,relu,False,100,sgd


In [635]:
clean_list = get_datasetwise_df(clean_sgd)

clean_res_dfs = get_f1accs(clean_list)
clean_res = {}
for i in range(len(dataset_list)):
    clean_res[dataset_list[i]] = clean_res_dfs[i].to_dict()
pd.DataFrame(clean_res).T

,Vanilla test acc,Vanilla test f1,Shadow test acc,Shadow test f1,MIA mlp,MIA svm,MIA ranfor,MIA logi,MIA ada,MIA confidence mse,Vanilla runtime per
cora,0.8112,0.8112,0.8128,0.8128,0.5821,0.5936,0.5665,0.5468,0.5698,0.6026,0.1845
citeseer,0.7219,0.7219,0.7380,0.7380,0.5926,0.5813,0.5739,0.5505,0.5764,0.5952,0.1805
pubmed,0.6105,0.6105,0.5940,0.5940,0.5000,0.5050,0.5187,0.5108,0.5074,0.5123,0.1903
computers,0.8251,0.8251,0.8229,0.8229,0.5130,0.5158,0.5112,0.5193,0.5143,0.5192,0.1857
photo,0.9140,0.9140,0.9198,0.9198,0.5192,0.5192,0.5186,0.5221,0.5180,0.5299,0.1812
cs,0.8312,0.8312,0.8303,0.8303,0.5155,0.5170,0.5093,0.5171,0.5138,0.5139,0.1938
physics,0.9343,0.9343,0.9305,0.9305,0.5103,0.5132,0.5054,0.5129,0.5096,0.5078,0.1977
reddit,0.6953,0.6953,0.6897,0.6897,0.5077,0.5141,0.5081,0.5121,0.5112,0.5131,0.2187
github,0.8582,0.8582,0.8592,0.8592,0.5058,0.5083,0.5086,0.5068,0.5093,0.5107,0.1815
flickr,0.4596,0.4596,0.4668,0.4668,0.5059,0.5125,0.5060,0.5068,0.5095,0.5370,0.1908


In [637]:
def get_res_rdp(cora:pd.DataFrame):
    maxx = ['Vanilla test acc','Vanilla test f1']
    meann = ['Vanilla runtime per','MIA mlp', 'MIA svm','MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Epsilon','Delta']
    minn = ['Epsilon','Delta']
    res = {}
   
    for k in maxx:
        res[k] = cora.max()[k]
    for k in meann:
        res[k] = cora.mean(numeric_only=True)[k]
    for k in minn:
        res[k] = cora.mean(numeric_only=True)[k]
    return res
    
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cora_ns1_nb1_cluster.csv',index_col=0)
df

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type
0,cora,0.8670,0.5521,0.8670,0.7094,0.9923,0.7094,1.0000,0.0125,1.0000,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
1,cora,0.8522,0.6419,0.8522,0.7028,1.0155,0.7028,1.0000,0.0125,1.0000,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
2,cora,0.8489,0.7091,0.8489,0.7291,0.9462,0.7291,0.9984,0.0128,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
3,cora,0.8177,0.7117,0.8177,0.7011,0.9737,0.7011,0.9901,0.0271,0.9901,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
4,cora,0.8637,0.6601,0.8637,0.7521,0.9392,0.7521,0.9984,0.0102,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
5,cora,0.8522,0.6768,0.8522,0.7274,0.9609,0.7274,0.9967,0.0167,0.9967,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
6,cora,0.8489,0.6410,0.8489,0.7094,0.9636,0.7094,0.9984,0.0133,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
7,cora,0.8440,0.6866,0.8440,0.7110,0.9748,0.7110,0.9984,0.0144,0.9984,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
8,cora,0.8079,0.7155,0.8079,0.6585,1.0695,0.6585,0.9967,0.0127,0.9967,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam
9,cora,0.8522,0.6521,0.8522,0.7209,0.9416,0.7209,0.9967,0.0166,0.9967,...,0.45,2,256,0.001,0.001,0.5,relu,True,200,adam


In [638]:
get_res_rdp(df)

{'Vanilla test acc': 0.7521,
 'Vanilla test f1': 0.7521,
 'Vanilla runtime per': 0.21726363636363633,
 'MIA mlp': 0.5046272727272727,
 'MIA svm': 0.5177545454545456,
 'MIA ranfor': 0.5016454545454546,
 'MIA logi': 0.5147,
 'MIA ada': 0.5016363636363635,
 'MIA confidence mse': 0.5863636363636363,
 'Epsilon': 86.9397,
 'Delta': 0.0033}